In [8]:
import numpy as np



# Example 2D NumPy array
arr = np.array([[1, 2, 3],
                [4, 5, 6],
                [7, 8, 9]])

# Get the shape of the array
original_shape = arr.shape

# Pad out the array with the final column
padded_arr = np.pad(arr, ((0, 1), (0, 0)), mode='constant')

print(original_shape[0])
print(padded_arr.shape[0])

print("Original Array:")
print(arr)
print("Padded Array:")
print(padded_arr)

for i, action in enumerate(arr):
    print(action)
    print(padded_arr[i,:])

3
4
Original Array:
[[1 2 3]
 [4 5 6]
 [7 8 9]]
Padded Array:
[[1 2 3]
 [4 5 6]
 [7 8 9]
 [0 0 0]]
[1 2 3]
[1 2 3]
[4 5 6]
[4 5 6]
[7 8 9]
[7 8 9]


In [1]:
esp32 = "http://192.168.1.182"

import requests



In [2]:
import os
import numpy as np
import cv2
import h5py
import argparse
import time
from visualize_episodes import visualize_joints, visualize_timestamp, save_videos

import matplotlib.pyplot as plt
from constants import DT

import IPython
e = IPython.embed

JOINT_NAMES = ["waist", "shoulder", "elbow", "forearm_roll", "wrist_angle", "wrist_rotate"]
STATE_NAMES = JOINT_NAMES + ["gripper"]

MIRROR_STATE_MULTIPLY = np.array([-1, 1, 1, -1, 1, -1, 1]).astype('float32')
MIRROR_BASE_MULTIPLY = np.array([1, -1]).astype('float32')

def load_hdf5(dataset_dir, dataset_name):
    dataset_path = os.path.join(dataset_dir, dataset_name + '.hdf5')
    if not os.path.isfile(dataset_path):
        print(f'Dataset does not exist at \n{dataset_path}\n')
        exit()

    with h5py.File(dataset_path, 'r') as root:
        is_sim = root.attrs['sim']
        compressed = root.attrs.get('compress', False)
        qpos = root['/observations/qpos'][()]
        # qvel = root['/observations/qvel'][()]
        action = root['/action'][()]
        image_dict = dict()
        for cam_name in root[f'/observations/images/'].keys():
            image_dict[cam_name] = root[f'/observations/images/{cam_name}'][()]
        if 'base_action' in root.keys():
            print('base_action exists')
            base_action = root['/base_action'][()]
        else:
            base_action = None
        if compressed:
            compress_len = root['/compress_len'][()]

    if compressed:
        for cam_id, cam_name in enumerate(image_dict.keys()):
            # un-pad and uncompress
            padded_compressed_image_list = image_dict[cam_name]
            image_list = []
            for padded_compressed_image in padded_compressed_image_list: # [:1000] to save memory
                image = cv2.imdecode(padded_compressed_image, 1)
                image_list.append(image)
            image_dict[cam_name] = np.array(image_list)

    return qpos, action, base_action, image_dict, is_sim

In [4]:
data_dir = "/home/jonny/projects/mobile_aloha_world/episodes/task1"
dataset_name = 'episode_1'

In [7]:
qpos,  action, base_action, image_dict, is_sim = load_hdf5(data_dir, dataset_name)

        # process proprioception
qpos = np.array(qpos).astype('int32') #np.concatenate([qpos[:, 7:] * MIRROR_STATE_MULTIPLY, qpos[:, :7] * MIRROR_STATE_MULTIPLY], axis=1)
# qvel = np.concatenate([qvel[:, 7:] * MIRROR_STATE_MULTIPLY, qvel[:, :7] * MIRROR_STATE_MULTIPLY], axis=1)
action = np.array(action).astype('int32')  #np.concatenate([action[:, 7:] * MIRROR_STATE_MULTIPLY, action[:, :7] * MIRROR_STATE_MULTIPLY], axis=1)
if base_action is not None:
    base_action = np.array(base_action).astype('int32') #* MIRROR_BASE_MULTIPLY
    while (base_action.shape[0] < action.shape[0]):
        #pad out array
        base_action=np.pad(base_action, ((0, 1), (0, 0)), mode='constant')

base_action exists


In [8]:
#change from rgb to bgr
image_dict['top'] = image_dict['top'][:, :, ::-1]
image_dict['base'] = image_dict['base'][:, :, ::-1]


In [10]:
data_dict = {
    '/observations/qpos': qpos,
    # '/observations/qvel': qvel,
    '/action': action,
    '/base_action': base_action,
} if base_action is not None else {
    '/observations/qpos': qpos,
    # '/observations/qvel': qvel,
    '/action': action,
}
for cam_name in image_dict.keys():
    if 'base' in image_dict.keys():
        data_dict[f'/observations/images/{cam_name}'] = image_dict[cam_name]
max_timesteps = len(qpos)

COMPRESS = True


In [24]:
t0 = time.time()
encode_param = [int(cv2.IMWRITE_JPEG_QUALITY), 50] # tried as low as 20, seems fine
compressed_len = []
for cam_name in image_dict.keys():
    image_list = data_dict[f'/observations/images/{cam_name}']
    print(len(image_list))
    compressed_list = []
    compressed_len.append([])
    for image in image_list:
        result, encoded_image = cv2.imencode('.jpg', image, encode_param) # 0.02 sec # cv2.imdecode(encoded_image, 1)
        compressed_list.append(encoded_image)
        compressed_len[-1].append(len(encoded_image))
    data_dict[f'/observations/images/{cam_name}'] = compressed_list
print(f'compression: {time.time() - t0:.2f}s')

# pad so it has same length
t0 = time.time()


146
146
compression: 0.36s


In [21]:
print(compressed_len)

[[5625, 5599, 5624, 5609, 5563, 5362, 5599, 5804, 5588, 5791, 6029, 5721, 5844, 6083, 6045, 6084, 6089, 5933, 6059, 6070, 6057, 6095, 6181, 6174, 6096, 6246, 6057, 6016, 6134, 6141, 6079, 6049, 6159, 6373, 6355, 6867, 6859, 6911, 6916, 7143, 7153, 7195, 7111, 7017, 6825, 6784, 6913, 6823, 6789, 6646, 6725, 8159, 8280, 8323, 8367, 8383, 8497, 9173, 9222, 8966, 9162, 8147, 7413, 6852, 5907, 5047, 5061, 4916, 5087, 5092, 5101, 4914, 5009, 4938, 5136, 4942, 5083, 5124, 4964, 5115, 5114, 5133, 5002, 7331, 9316, 9447, 9215, 9152, 9145, 8089, 7743, 7058, 7615, 7476, 7917, 7864, 7928, 7599, 7155, 7612, 7847, 7737, 8001, 8065, 8053, 8153, 8277, 8189, 7791, 7772, 7953, 8045, 8052, 8184, 8207, 7672, 7364, 7198, 6545, 6518, 6517, 6431, 6528, 6352, 6341, 6294, 6468, 6506, 6542, 6486, 6477, 6017, 5904, 6043, 6164, 5747, 5935, 5939, 6038, 5996, 5977, 5984, 5987, 5948, 6027, 5992], [48206, 48359, 48081, 48083, 48172, 48766, 47008, 48065, 47097, 46591, 46712, 46461, 46566, 46593, 45769, 46507, 46183, 4

In [23]:
compressed_len = np.array(compressed_len)
print(compressed_len)
padded_size = compressed_len.max()
print(padded_size)

[[ 5625  5599  5624  5609  5563  5362  5599  5804  5588  5791  6029  5721
   5844  6083  6045  6084  6089  5933  6059  6070  6057  6095  6181  6174
   6096  6246  6057  6016  6134  6141  6079  6049  6159  6373  6355  6867
   6859  6911  6916  7143  7153  7195  7111  7017  6825  6784  6913  6823
   6789  6646  6725  8159  8280  8323  8367  8383  8497  9173  9222  8966
   9162  8147  7413  6852  5907  5047  5061  4916  5087  5092  5101  4914
   5009  4938  5136  4942  5083  5124  4964  5115  5114  5133  5002  7331
   9316  9447  9215  9152  9145  8089  7743  7058  7615  7476  7917  7864
   7928  7599  7155  7612  7847  7737  8001  8065  8053  8153  8277  8189
   7791  7772  7953  8045  8052  8184  8207  7672  7364  7198  6545  6518
   6517  6431  6528  6352  6341  6294  6468  6506  6542  6486  6477  6017
   5904  6043  6164  5747  5935  5939  6038  5996  5977  5984  5987  5948
   6027  5992]
 [48206 48359 48081 48083 48172 48766 47008 48065 47097 46591 46712 46461
  46566 46593 45769 465

In [14]:
len(encoded_image)

30403

In [19]:
tmp=[]
tmp.append([])
print(tmp)
tmp[-1].append(3)
print(tmp)

[[]]
[[3]]
